### In this file we will use  the techniques to increase the model performance and accuracy.  

### Keeping the outlier.  
We will be using some of the HyperTunning techiniques to traing the model.  
1st we will use Ordinal Ecoding.  
2nd we will use OneHotEncoder to convert the categorical data.  

### Removing the Outliers.  
1st we will use Ordinal Ecoding.  
2nd we will use OneHotEncoder to convert the categorical data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import os
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
## Model Training
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.svm import LinearSVC

In [2]:
path=os.path.join("Data","flight_dataset.csv")
path

'Data/flight_dataset.csv'

In [3]:
data=pd.read_csv(path)
df=data.copy()

In [4]:
def convert_date_format(Data,Column):
    """
    This function is used to convert the given date format to a correct format.
    Complete date column will be splited into saperate columns
    Date, Month, Year this are the new columns to be created.
    """ 

    Data[Column]=pd.to_datetime(Data[Column],infer_datetime_format=True)
    Data['Day']=Data[Column].dt.day
    Data['Month']=Data[Column].dt.month
    Data['Year']=Data[Column].dt.year

In [5]:
convert_date_format(df,"Date_of_Journey")

/tmp/ipykernel_399/3225999564.py:8: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  Data[Column]=pd.to_datetime(Data[Column],infer_datetime_format=True)
/tmp/ipykernel_399/3225999564.py:8: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  Data[Column]=pd.to_datetime(Data[Column],infer_datetime_format=True)


In [6]:
def convert_time_format(Data,deptime,arivaltime):
    
    """
    This function convert th 24:00 Hr time format to 2 saperate index hour and min.
    New columns with Departure Hour, Arival Hour, Departure Min and Arival Min will be created.
    This function can be used when the time is in 00:00 this format. other format will not work(ex 00.00)
    """ 
    
    Data["Dep_hour"]=Data[deptime].str.split(":").str[0]
    Data["Dep_min"]=Data[deptime].str.split(":").str[1]

    Data[arivaltime]=Data[arivaltime].str.split(" ").str[0]
    Data["Arival_hour"]=Data[arivaltime].str.split(":").str[0]
    Data["Arival_min"]=Data[arivaltime].str.split(":").str[1]

In [7]:
convert_time_format(df,"Dep_Time","Arrival_Time") 

In [8]:
df[df["Duration"] == "5m"]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Day,Month,Year,Dep_hour,Dep_min,Arival_hour,Arival_min
6474,Air India,2019-03-06,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327,6,3,2019,16,50,16,55


In [9]:
df.drop(6474,axis=0,inplace=True)

In [10]:
def convert_travel_time(Data,DurationTime):
    """
    This function is used to conver the travelling time to saperate columns.
    Function will create 2 columns travelling hour and travelling min.
    """ 

    Data[DurationTime]=Data[DurationTime].str.split(" ")

    Data["Trveling_hour"]=Data[DurationTime].str[0].str.split("h").str[0]
    Data["Trveling_min"]=Data[DurationTime].str[1].str.split("m").str[0]

In [11]:
convert_travel_time(df,"Duration")

In [12]:
# converting the categorical value to numerical values.
dic={'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4}
df["Total_Stops"]=df["Total_Stops"].map(dic)

In [13]:
df.drop(labels=["Date_of_Journey","Route","Dep_Time","Arrival_Time","Duration"],axis=1,inplace=True)

In [14]:
# removing the duplicates and Trujet data and nan values.
print(f"The shape of data before removing duplicates is : {df.shape}")
df.drop_duplicates(inplace=True)
df.drop(2878,axis=0,inplace=True)
print(f"The shape of data after removing duplicates is : {df.shape}")

The shape of data before removing duplicates is : (10682, 15)
The shape of data after removing duplicates is : (10459, 15)


In [15]:
df["Trveling_min"]=df["Trveling_min"].fillna(0)
df["Total_Stops"]=df["Total_Stops"].fillna(0)

In [16]:
# Converting the object type to Integer type.
df["Dep_hour"] = df["Dep_hour"].astype(int)
df["Dep_min"] = df["Dep_min"].astype(int)
df["Arival_hour"] = df["Arival_hour"].astype(int)
df["Arival_min"] = df["Arival_min"].astype(int)
df["Trveling_hour"] = df["Trveling_hour"].astype(int)
df["Trveling_min"] = df["Trveling_min"].astype(int)
df["Total_Stops"] = df["Total_Stops"].astype(int)

In [17]:
# Split the columns to independent and dependent columns.
X = df.drop(labels="Price",axis=1)
Y = df[["Price"]]

In [18]:
# Split the columns to categorical and numerical columns.
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [19]:
from sklearn.preprocessing import OneHotEncoder

In [20]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OneHotEncoder(handle_unknown='ignore')),
    ('scaler',StandardScaler(with_mean=False))
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])

In [21]:
# Split the data to training and testing.
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [22]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [23]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [24]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier